## Load Packages

In [1]:
# Link to Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install geopandas
!pip install rioxarray
!pip install geojson
!pip install rasterstats

import os
from glob import glob
import numpy as np
import pandas as pd
import geojson
import shapely as shp
from shapely.geometry import Polygon, box
import geopandas as gpd
import xarray as xr
import rioxarray as rioxr
from rasterstats import zonal_stats
import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 1.0 MB 21.6 MB/s 
     |████████████████████████████████| 6.3 MB 44.0 MB/s 
     |████████████████████████████████| 15.4 MB 48.3 MB/s 
     |████████████████████████████████| 47 kB 3.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.3 MB 1.4 MB/s 
  Created wheel for rioxarray: filename=rioxarray-0.9.1-py3-none-any.whl size=54611 sha256=ecd85f1f949304267c4f358f9eb787b0e18c82cb60431ca7e179d5bf5360e288
  Stored in directory: /root/.cache/pip/wheels/07/da/9e/1cc57b2e7a29a206893db83e984a341e2e94378263e0798229
Successfully built rioxarray
     |████████████████████████████████| 130 kB 28.5 MB/s 


## Zonal Stats Processing Function

### User Input

In [23]:
# Directory "Images"
path_img = '/content/drive/My Drive/myExportImage/test_data_for_marco/Img_waterMsk/'
Images = glob(os.path.join(path_img, '*.tif'))

# Directory "Ponds"
path_ponds = '/content/drive/My Drive/myExportImage/test_data_for_marco/ponds_by_parcel/'

In [24]:
# Output Path
outputPath_zonstatsDF = '/content/drive/My Drive/THESIS_AQUAPONDS/gdf_zonstats'

### Global Setting & Functions

In [25]:
# Pair up Index and Years: In order to assign matched year to each xr.dataArray 
keys = [str(i) for i in range(36)]
#years = pd.date_range(start='1984', end='2020', freq='Y')
years = [i for i in range(1984,2020)]
pair = dict([[i,j] for i,j in zip(keys, years)])
print(pair)

{'0': 1984, '1': 1985, '2': 1986, '3': 1987, '4': 1988, '5': 1989, '6': 1990, '7': 1991, '8': 1992, '9': 1993, '10': 1994, '11': 1995, '12': 1996, '13': 1997, '14': 1998, '15': 1999, '16': 2000, '17': 2001, '18': 2002, '19': 2003, '20': 2004, '21': 2005, '22': 2006, '23': 2007, '24': 2008, '25': 2009, '26': 2010, '27': 2011, '28': 2012, '29': 2013, '30': 2014, '31': 2015, '32': 2016, '33': 2017, '34': 2018, '35': 2019}


In [26]:
# Parcel IDs
#parcelIDs = sorted(list(set([i.split('/')[-1].split('_')[1] for i in Images])))
parcelIDs = sorted(list(set([i.split('/')[-1].split('.')[0].split('-')[1] for i in Images])))

print('Parcel IDs:', parcelIDs)
len(parcelIDs)

Parcel IDs: ['215', '482', '517', '567', '601', '640', '667', '689', '692']


9

In [27]:
def tidy_zonstats_df(zonstats_dict, gdf, year):
  # Tidy dataframe of zonal statistics
  df_stats = pd.DataFrame(zonstats_dict)

  if 0 not in df_stats:
    df_stats[0] = np.nan
  if 1 not in df_stats:
    df_stats[1] = np.nan
  if 2 not in df_stats:
    df_stats[2] = np.nan

  # Convert values to proper data type
  df_stats = df_stats.convert_dtypes()
  # Rename columns
  colname_c0 = 'c0_'+str(year)
  colname_c1 = 'c1_'+str(year)
  colname_c2 = 'c2_'+str(year)
  df_stats.rename(columns={0:colname_c0, 1:colname_c1, 2:colname_c2}, inplace=True)
  # Add 'pondID' as new column
  df_stats = df_stats.assign(pondID = np.array(gdf['pondID']))
  df_stats[[colname_c0, colname_c1, colname_c2]] = df_stats[[colname_c0, colname_c1, colname_c2]].fillna(0)
  return df_stats



In [28]:
def zonal_statistics(year, gdf, xds, affine):

  # Subset raster to one year
  xda_1Y = xds.sel(time=year).to_array()
  # Convert DataArray to NumpyArray
  array = xda_1Y.values[0]

  # 1st Run Zonal statistics
  zonstats_1st = zonal_stats(gdf, array, transform=affine, categorical=True, all_touched=False) 

  # Tidy dataframe of zonal statistics
  df_stats = tidy_zonstats_df(zonstats_1st, gdf, year)

  # Store needed Stats from Run 1 and Fetch Ponds for Run 2
  colname_c0 = 'c0_'+str(year)
  colname_c1 = 'c1_'+str(year)
  colname_c2 = 'c2_'+str(year)
  stats_run1 = df_stats[(df_stats[colname_c0]!=0) | (df_stats[colname_c1]!=0) | (df_stats[colname_c2]!=0)]
  ponds_run2 = gdf.merge(df_stats[(df_stats[colname_c0]==0) & (df_stats[colname_c1]==0) & (df_stats[colname_c2]==0)], how='right', on='pondID')

  if ponds_run2.empty:

    zonstats = stats_run1

  else:
    
    # 2nd Run Zonal Stats
    zonstats_2nd = zonal_stats(ponds_run2, array, transform=affine, categorical=True, all_touched=True)

    # Tidy dataframe of zonal statistics
    stats_run2 = tidy_zonstats_df(zonstats_2nd, ponds_run2, year) 

    # Join Outputs
    zonstats = pd.concat([stats_run1, stats_run2])

  return zonstats

In [29]:
def zonstats_by_parcel(parcelID):
  # Ponds of Parcel_OO
  pondItem = glob(os.path.join(path_ponds, '*'+parcelID+'*'))
  ponds = gpd.read_file(pondItem[0])
  ponds.rename(columns={'id':'pondID'}, inplace=True)

  # Images of Parcel_OO
  imageItems = glob(os.path.join(path_img, '*'+parcelID+'*.tif'))

  # Tidy Images of Parcel_OO
  elements = [rioxr.open_rasterio(i) for i in imageItems]
  xdas = [i.rename({'band':'time'}) for i in elements]
  xdss = [i.to_dataset(name='watermask') for i in xdas]
  xdss = [i.reindex(y = list(reversed(i.y))) for i in xdss]
  # Get the index of loaded image
  imgID = [i.split('_')[0] for i in xdas[0].attrs['long_name']]
  # Get years based on index
  t = [pair[i] for i in imgID]
  for i in range(len(xdss)):
    xdss[i].coords['time'] = np.array(t)

  # "Affine Transformations" of Images of Parcel_OO
  transform = [i.rio.transform() for i in xdss]
  we_pr, rotation, ns_pr = [i[0] for i in transform], [i[1] for i in transform], [i[4] for i in transform]
  xmin, xmax, ymin, ymax = [i.x.min().values for i in xdss], [i.x.max().values for i in xdss], [i.y.min().values for i in xdss], [i.y.max().values for i in xdss]
  affines = [[a,b,c,d,e,f] for a,b,c,d,e,f in zip(xmin, we_pr, rotation, ymax, rotation, ns_pr)]

  # Bounding Boxes of Images of Parcel_OO
  bbox = [box(minx, miny, maxx, maxy) for minx, miny, maxx, maxy in zip(xmin, ymin, xmax, ymax)]

  # Tiled Ponds of Parcel_OO
  ponds_sub = [gpd.GeoDataFrame(ponds[ponds.geometry.within(i)]) for i in bbox]

  # Remove Items from "tiled ponds", "tidy images" and "affines" where the ponds subset is empty.
  index_to_keep = [k for k,i in enumerate(ponds_sub) if i.empty!=True]
  # Filtered Ponds Tiles of Parcel_OO
  ponds_input = [i for k,i in enumerate(ponds_sub) if k in index_to_keep]
  # Filtered Tidy Images of Parcel_OO
  xdss_input = [i for k,i in enumerate(xdss) if k in index_to_keep]
  # Filtered Affines of Parcel_OO
  affines_input = [i for k,i in enumerate(affines) if k in index_to_keep]

  # Small Loop (over ponds tiles of Parcel_OO)

  ### Reduced Time Periods & Reduced Ponds Amount
  years = t[-2:]
  zonstats_listNested = [[zonal_statistics(year=i, gdf=ponds_input[j].iloc[:5], xds=xdss_input[j], affine=affines_input[j]) for i in years] for j in range(len(ponds_input))]
  zonstats_listFlat = [pd.concat([df.set_index('pondID') for df in j], axis=1) for j in zonstats_listNested]
  ponds_zonstats_list = [i.iloc[:5].merge(j, how='outer', on='pondID') for i,j in zip(ponds_input, zonstats_listFlat)]

  ### Whole Time Period & All Ponds of Parcel_OO
  #years = t
  #zonstats_listNested = [[zonal_statistics(year=i, gdf=ponds_input[j], xds=xdss_input[j], affine=affines_input[j]) for i in years] for j in range(len(ponds_input))]
  #zonstats_listFlat = [pd.concat([df.set_index('pondID') for df in j], axis=1) for j in zonstats_listNested]
  #ponds_zonstats_list = [i.merge(j, how='outer', on='pondID') for i,j in zip(ponds_input, zonstats_listFlat)]


  # df zonStats of Parcel_OO
  ponds_zonstats = pd.concat([df for df in ponds_zonstats_list])
  return ponds_zonstats

In [30]:
def genFun(n): # n = len(parcelIDs)
  i = 0
  while i < n:
    parcelID = parcelIDs[i]
    ponds_zonstats = zonstats_by_parcel(parcelID)
    yield ponds_zonstats.to_file(os.path.join(outputPath_zonstatsDF, 'zonstats_p_'+str(parcelID)+'.geojson'), driver='GeoJSON')
    i += 1

gen = genFun(len(parcelIDs))

In [31]:
# Exhaust Generator
# Reference: https://stackoverflow.com/questions/47456631/simpler-way-to-run-a-generator-function-without-caring-about-items
from collections import deque

def exhaust(generator):
    deque(generator, maxlen=0)

exhaust(gen)